In [1]:
import re
# from flaml import tune
from matplotlib import rcParams
from function2 import *
import glob
import lightgbm as lgb
import warnings
from sklearn.model_selection import GridSearchCV
import pymssql
import os
import joblib
import shutil
import numpy as np
import pandas as pd
import pyodbc
warnings.filterwarnings("ignore")
# from flaml import AutoML
# from flaml.default import LGBMRegressor
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
# rcParams['font.family'] = 'SimHei'
from sql_server_token import PASSWORD

### 基础参数设置

In [2]:
'''训练模型'''
# 注：下文中，测试集指一般意义上的验证集，验证集指一般意义下的测试集
time_unit = '月'   # 数据长度单位
unit = '周'   # 收益频率
n_train = 36    #训练集长度
response_type = '收益率' #可以选择收益率，夏普率，收益率分类，夏普率分类作为标签，目前使用收益率；若使用分类标签，后续训练过程有参数需要改变
rolling_window = 1  # rolling_window一般不需改动
# rolling_unit = '月' # 可选年/月/周
get_date_unit = '年' # get_date的间隔参数 测试集长度
get_date_unit_validation = '月' # validation_set的参数 可选年/月/half(half year)

initial_year = 2019 # 回测起始年
final_year = 2021 #回测结束年
data_local = False # 数据是否已存本地 第一次运行需要False，后续改为True
data_in_all = True #股票池是否来自全市场
OutputPath_result = 'model_output_all' # 最终结果的存储路径
result_excel = 'output_3_year_train_total.xlsx' # 最终结果的文件名
y_predict_name = 'y_by_year_all.pkl' # 因子预测值储存文件名
validation_set = 1
rolling_style = 1
orthogonalization = True #是否正交化
factor_to_zero = False #是否将部分因子值设为0



# shutil.rmtree('model_regression_500_best_' + str(int((n_train)/12))+'year_train_by_month')
# os.mkdir('model_regression_500_best_' + str(int((n_train)/12))+'year_train_by_month')


# param_grid = {
#    'max_depth':  [3,4,5],
#    'learning_rate': [0.05,0.1,0.2],
#    'n_estimators': [100,200,300,400,500,600]
# }


param_grid = {
    'max_depth':  [3],
    'learning_rate': [0.01],
    'n_estimators': [100]
}


file_names = sorted(glob.glob(r'C:\Users\24343\Desktop\实习\国泰君安量化组\数据库\因子数据\因子中性化数据1130\*'))   # 加载文件名称
# file_names = sorted(glob.glob(r'C:\bak\新增資料夾\lgb因子\因子标准化数据1130.part01\因子标准化数据1130\*'))   # 加载文件名称

# dates是全程日期，dates_train是测试集取值的范围
dates_train = [datetime(year, 1, 1).strftime('%Y-%m-%d') for year in range(initial_year, final_year + 1)]

server = '(local)'
database = 'winddb0102'
username = 'sa'

# conn = pymssql.connect('(local)', 'sa', '123456', 'winddb0102')  # 连接sql server
conn = pyodbc.connect(f'DRIVER={{SQL Server}};'
                      f'SERVER={server};'
                      f'DATABASE={database};'
                      f'UID={username};'
                      f'PWD={PASSWORD}'
)

### 先获取总交易日，然后取出月/周来训练: 先取三年，然后取一个月/周回测，测完加入训练集，再删一个月/周，训练集长度保持不变

In [3]:
# 总交易日：
Calender_all = Load_Calender(file_names, dates_train[0], 12*len(dates_train), time_unit, '日')
Calender_all


,TRADE_DT
0,20190102
1,20190103
2,20190104
3,20190107
4,20190108
...,...
725,20211227
726,20211228
727,20211229
728,20211230


### 再获取周度调仓日

In [4]:
# 调仓日
Calender_change = get_trade_date(file_names, dates_train[0], '周')
Calender_change['TRADE_DT'] = pd.to_datetime(Calender_change['TRADE_DT'], format='%Y%m%d')
Calender_change

,TRADE_DT
0,2019-01-02
1,2019-01-09
2,2019-01-16
3,2019-01-23
4,2019-01-30
...,...
241,2023-12-19
242,2023-12-26
243,2024-01-03
244,2024-01-10


### 具体函数

In [5]:
def set_factors_to_zero(data):
    """
    处理数据集中的因子
    
    参数：
    - data: 包含时间、股票名称、因子和收益率的DataFrame
    
    返回值：
    - 更新后的data，其中因子列被替换为处理后的因子数据
    """
    # 提取因子数据（剔除前两列和最后一列）
    factor_data = data.iloc[:, 2:-1]

    # 计算因子中位数
    factor_medians = factor_data.median()

    # 使用NumPy进行向量化运算，将小于中位数的值设置为空
    processed_factor_data = np.where(factor_data.values < factor_medians.values, 0, factor_data.values)

    # 将原始data中的因子数据更新为处理后的因子数据
    data.iloc[:, 2:-1] = processed_factor_data
    
    return data


def normalization(data):
    # 提取因子数据（剔除前两列和最后一列）
    factor_data = data.iloc[:, 2:-1]
    
    # 对因子数据进行横截面z-score归一化
    normalized_factors = (factor_data - factor_data.mean()) / factor_data.std()
    data.iloc[:, 2:-1] = normalized_factors
    # data = data.dropna(axis=0, how='any').reset_index(drop=True) #需无空列

    return data




def orthogonalize_factors(data): #无空列
    """
    将训练集中的因子进行对称正交
    
    参数：
    - data: 包含时间、股票名称、因子和收益率的DataFrame
    
    返回值：
    - 更新后的data，其中因子列被替换为对称正交后的因子数据
    """
    data = normalization(data)
    normalized_factors = data.iloc[:, 2:-1]

    # 计算特征值和特征向量
    D, U = np.linalg.eig(np.dot(normalized_factors.T, normalized_factors))

    D = np.abs(D) #取绝对值防止报错
    U = np.mat(U)
    d = np.diag(D**(-0.5))
    S = (U*d*U.T).real #取实部防止报错
    F_hat = np.mat(normalized_factors)*S 
    
    # 将data中的因子数据更新为正交后的因子数据
    data.iloc[:, 2:-1] = F_hat
    
    return data


def orthogonalize_factors_with_nan(data):
    """
    根据输入数据的因子情况，进行对称正交
    
    参数：
    - data: 包含时间、股票名称、因子和收益率的DataFrame
    
    返回值：
    - 更新后的data，其中因子列被替换为对称正交后的因子数据
    """

    # 检查是否存在空列
    if (data == 0).all().any():
        # 存在空列，记下空列的位置和字段
        zero_columns = data.columns[(data == 0).all()].tolist()
        # 删除空列
        result_data = data.drop(zero_columns, axis=1)

        
        # 进行正交处理
        result_data = orthogonalize_factors(result_data)

        # 在原来空列的位置上添加空列
        for col in zero_columns:
            # 获取原空列的位置
            col_position = data.columns.get_loc(col)
            # 在原位置上插入空列
            result_data.insert(col_position, col, np.nan)

    else:
        # 不存在空列，直接使用正交函数更新数据
        result_data = orthogonalize_factors(data)

    return result_data


### 数据集初始化

In [6]:
initial_date = Calender_all.TRADE_DT[0]
# initial_date = '-'.join([initial_date[:4], initial_date[4:6], initial_date[6:]])
Calender_train = Load_Calender(file_names, initial_date, n_train, time_unit, '日', date_begin=False)  # 加载初始训练集日期
data_files_train = filter_file_name(file_names, Calender_train)
    
Index_date = Load_Index_Weights(conn, Calender_train)
Index_date['TRADE_DT'] = pd.to_datetime(Index_date['TRADE_DT'], format='%Y%m%d')
Index_date['month'] = Index_date['TRADE_DT'].dt.to_period('M')


# 训练集要改 两种 对应
train_data = read_data(data_files_train, response_type, unit, if_split = False)  #初始 全市场 训练集
train_data['TRADE_DT'] = pd.to_datetime(train_data['TRADE_DT'], format='%Y%m%d')


if orthogonalization:
    train_data = train_data.groupby('TRADE_DT').apply(orthogonalize_factors_with_nan).reset_index(drop=True)

if factor_to_zero:
    train_data = train_data.groupby('TRADE_DT').apply(set_factors_to_zero).reset_index(drop=True)


train_data['last_month'] = (train_data['TRADE_DT'] - pd.DateOffset(months=1)).dt.to_period('M')  

data_500_list = []
for month in Index_date['month'].unique():
    code_500 = Index_date[Index_date['month'] == month]['S_INFO_WINDCODE']
    data_500_list.append(train_data[(train_data['last_month']==month)&(train_data['S_INFO_WINDCODE'].isin(code_500))])
        
data_500 = pd.concat(data_500_list, ignore_index=True) #初始 500成分股 训练集

del data_500['last_month']
del train_data['last_month']

data_500

读取数据...: 100%|██████████| 731/731 [05:44<00:00,  2.12it/s]


,TRADE_DT,S_INFO_WINDCODE,Amount_20D_AVG,Amt2RealizedVolatility_5D,BP,CFP_ttm,CashDividendPayoutRatio,EBIT2EV,EPFY3,EP_ded_qfa,...,or_fy3_tb_120d,rank_pbroe_qfa,rank_peg_qfa2,rounte_mom,rtd,turnover1m,turnover3m,turnover_std_3m,varTradeVolume5_10,zdf5
0,2016-01-04,000006.SZ,0.016521,-0.001878,-0.013350,-0.050224,0.039797,0.023656,-0.009673,-0.013850,...,0.003320,0.002295,-0.023689,-0.014387,0.008400,0.006687,-0.005394,-0.007455,-0.044167,-0.186898
1,2016-01-04,000012.SZ,0.018360,0.045949,-0.010283,0.008461,0.041792,-0.013154,-0.004473,-0.025269,...,0.018467,0.023287,0.032429,0.063063,-0.011320,0.003509,-0.017234,0.001372,-0.011703,-0.179700
2,2016-01-04,000021.SZ,0.007604,-0.009174,0.001322,-0.028544,-0.000120,-0.010430,-0.006332,0.025829,...,-0.001388,0.007405,-0.025029,-0.010256,0.013120,-0.008876,-0.008485,-0.015728,0.008078,-0.214747
3,2016-01-04,000028.SZ,-0.017967,-0.012962,-0.019874,-0.012101,-0.021628,0.006216,0.001504,0.012944,...,-0.009309,0.013404,0.011113,0.000605,-0.003714,0.024293,-0.024580,-0.012023,0.002113,0.000000
4,2016-01-04,000030.SZ,-0.025294,-0.003740,0.003846,0.003205,0.024752,0.005555,-0.012210,0.011224,...,0.002417,0.025218,0.011325,0.036130,0.037349,-0.001005,-0.009503,0.003532,-0.013646,-0.172638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365486,2018-12-28,603877.SH,-0.031773,-0.003502,-0.006375,0.002475,0.027758,0.005939,-0.001940,-0.004896,...,-0.001255,-0.000130,-0.001532,0.001229,-0.014368,-0.018003,-0.003988,-0.020923,-0.014804,-0.020224
365487,2018-12-28,603883.SH,-0.015326,0.027342,-0.007114,0.001327,0.019745,-0.008971,-0.011490,-0.003990,...,0.002026,-0.003639,-0.001235,-0.006579,0.011113,-0.012099,-0.007568,-0.005972,-0.039398,-0.036857
365488,2018-12-28,603885.SH,-0.015847,0.029671,-0.005167,0.003013,-0.003062,-0.001466,0.002157,0.006290,...,-0.026506,-0.021470,-0.014746,0.001025,0.013272,-0.010397,-0.010026,-0.005418,0.012436,-0.032721
365489,2018-12-28,603888.SH,-0.007538,0.006159,-0.016373,-0.001432,0.007393,-0.009854,0.001899,-0.012305,...,-0.001168,-0.002947,-0.008553,0.001808,-0.001674,0.002796,0.002225,-0.005413,0.003503,0.029595


In [7]:
date = initial_date
if data_in_all:
    train_set = train_data

else:
    train_set = data_500

if validation_set:

    Calender_append = get_next_date_validation(file_names, date, get_date_unit_validation)
    last_date = datetime.strptime(Calender_append.TRADE_DT.tolist()[-1], '%Y%m%d')
    date = next((day for day in Calender_all.TRADE_DT.tolist() if datetime.strptime(day, '%Y%m%d') > last_date), None)

    data_files_append = filter_file_name(file_names, Calender_append)


    data_append_validation = read_data(data_files_append, response_type, unit, if_split = False)  #添加 全市场数据
    data_append_validation['TRADE_DT'] = pd.to_datetime(data_append_validation['TRADE_DT'], format='%Y%m%d')

    if orthogonalization:
        data_append_validation = data_append_validation.groupby('TRADE_DT').apply(orthogonalize_factors_with_nan).reset_index(drop=True)

    if factor_to_zero:
        data_append_validation = data_append_validation.groupby('TRADE_DT').apply(set_factors_to_zero).reset_index(drop=True)


    if not data_in_all:
        # 添加的训练集属于   500成分股
        Index_date = Load_Index_Weights(conn, Calender_append)      # 对应添加 500成分股的 时间跨度
        Index_date['TRADE_DT'] = pd.to_datetime(Index_date['TRADE_DT'], format='%Y%m%d')
        Index_date['month'] = Index_date['TRADE_DT'].dt.to_period('M')

        data_append_validation['last_month'] = (data_append_validation['TRADE_DT'] - pd.DateOffset(months=1)).dt.to_period('M')

        data_append_list = []

        for month in Index_date['month'].unique():
            code_500 = Index_date[Index_date['month'] == month]['S_INFO_WINDCODE']
            data_append_list.append(data_append_validation[(data_append_validation['last_month'] == month) & (data_append_validation['S_INFO_WINDCODE'].isin(code_500))])

        data_append_validation = pd.concat(data_append_list, ignore_index=True)
        del data_append_validation['last_month']
        #会改变因子列的顺序吗？






#    validation_x = data_append_validation.iloc[:,:-1]
    # 从中取出属于换仓日的部分
#    validation_x = validation_x[validation_x['TRADE_DT'].isin(Calender_change['TRADE_DT'])]




读取数据...:   0%|          | 0/20 [00:00<?, ?it/s]

读取数据...: 100%|██████████| 20/20 [00:04<00:00,  4.72it/s]


### 滚动训练和预测：在做预测之前进行对换仓日的筛选

In [8]:

def find_best_model(param_grid,train_x,train_y,validation_x,validation_y):
    best_error = float('inf') #误差越小越好
    best_model = None
    i = 1
    for max_depth in param_grid['max_depth']: #网格搜索最佳参数
        for learning_rate in param_grid['learning_rate']:
            for n_estimators in param_grid['n_estimators']:
                print(f"This is {i} th round")
                params = {'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'num_leaves': 64,
                        'min_child_samples': 240,
                        'min_child_weight': 0.001,
                        'subsample': 0.8,
                        'colsample_bytree': 0.8,
                        'reg_alpha': 0,
                        'reg_lambda': 0,
                        'learning_rate': learning_rate,
                        'boosting_type': 'gbdt',
                        #                                   'early_stopping_round': 5,
                        'objective': 'regression',
                        'verbosity': 2
                        }  # 默认参数 
                lgb_model = lgb.LGBMRegressor(**params)
                lgb_model.fit(train_x.drop(['TRADE_DT', 'S_INFO_WINDCODE'], axis=1), train_y)

                test_y_validation = lgb_model.predict(validation_x.drop(['TRADE_DT', 'S_INFO_WINDCODE'], axis=1))
                error = mean_squared_error(validation_y, test_y_validation)
                if error < best_error:
                    best_model = lgb_model
                    best_error = error
                i += 1

    return best_error, best_model


test_y_predict_list = []



# expand a test_set first in the while loop
while date in Calender_all.TRADE_DT.tolist():
    # print('date' + date)
    # 接下来是对训练集的按周/月添加，同时作为新的一周/月，也是测试集
    # Calender_append = Load_Calender(file_names, date, rolling_window, rolling_unit, '日', date_begin=True)
    Calender_append = get_next_date(file_names, date, get_date_unit)
    last_date = datetime.strptime(Calender_append.TRADE_DT.tolist()[-1], '%Y%m%d')
    date = next((day for day in Calender_all.TRADE_DT.tolist() if datetime.strptime(day, '%Y%m%d') > last_date), None)

    data_files_append = filter_file_name(file_names, Calender_append) 
    
    
    data_append = read_data(data_files_append, response_type, unit, if_split = False)  #添加 全市场数据
    data_append['TRADE_DT'] = pd.to_datetime(data_append['TRADE_DT'], format='%Y%m%d')

    if orthogonalization:
        data_append = data_append.groupby('TRADE_DT').apply(orthogonalize_factors_with_nan).reset_index(drop=True)

    if factor_to_zero:
        data_append = data_append.groupby('TRADE_DT').apply(set_factors_to_zero).reset_index(drop=True)


    if not data_in_all:
        # 添加的训练集属于   500成分股
        Index_date = Load_Index_Weights(conn, Calender_append)      # 对应添加 500成分股的 时间跨度
        Index_date['TRADE_DT'] = pd.to_datetime(Index_date['TRADE_DT'], format='%Y%m%d')
        Index_date['month'] = Index_date['TRADE_DT'].dt.to_period('M')

        data_append['last_month'] = (data_append['TRADE_DT'] - pd.DateOffset(months=1)).dt.to_period('M')   
        
        data_append_list = []

        for month in Index_date['month'].unique():
            code_500 = Index_date[Index_date['month'] == month]['S_INFO_WINDCODE']
            data_append_list.append(data_append[(data_append['last_month'] == month) & (data_append['S_INFO_WINDCODE'].isin(code_500))])
        
        data_append = pd.concat(data_append_list, ignore_index=True)
        del data_append['last_month']  
        #会改变因子列的顺序吗？ 




    train_x = train_set.iloc[:,:-1]
    train_y = train_set.iloc[:,-1]
    test_x = data_append.iloc[:,:-1]

    # # 从中取出属于换仓日的部分
    test_x = test_x[test_x['TRADE_DT'].isin(Calender_change['TRADE_DT'])]
    # test_y = data_append.iloc[:,-1]


    # 训练模型
    start = train_set.TRADE_DT[0].strftime('%Y-%m-%d')
    end = train_set.TRADE_DT.iloc[-1].strftime('%Y-%m-%d')   
    
    print('训练从' + start + '到' + end + '的模型') ##这是 test setting



    if validation_set:
        validation_x = data_append_validation.iloc[:,:-1]
        validation_y = data_append_validation.iloc[:,-1]


    ###### add validation set
        
        start = data_append_validation.TRADE_DT.iloc[0].strftime('%Y-%m-%d')
        end = data_append_validation.TRADE_DT.iloc[-1].strftime('%Y-%m-%d') 
        print('训练从' + start + '到' + end + '的 valid 模型')
        best_error, best_model = find_best_model(param_grid,train_x,train_y,validation_x,validation_y)

        # 清内存
        del validation_x, validation_y ## delete here?
    else:
        best_error, best_model = find_best_model(param_grid,train_x,train_y,train_x,train_y) #无验证集就把训练集当验证集


    start = data_append.TRADE_DT[0].strftime('%Y-%m-%d')
    end = data_append.TRADE_DT.iloc[-1].strftime('%Y-%m-%d')
    
    print('test set 预测从' + start + '到' + end + '的模型')
    test_y_predict = Get_FACTOR_Neu(test_x, best_model, if_regression = True)
    test_y_predict_list.append(test_y_predict)

   
    if validation_set:
        # merge validationn set and test set
        valid_and_test = pd.concat([data_append_validation,data_append], ignore_index=True)
        # front_set and back_set
        Calender_front = get_next_date(file_names, data_append_validation.TRADE_DT.iloc[0].strftime('%Y%m%d'), get_date_unit)
        Calender_front['TRADE_DT'] = pd.to_datetime(Calender_front['TRADE_DT'], format='%Y%m%d')
        front_set = valid_and_test[valid_and_test['TRADE_DT'].isin(Calender_front['TRADE_DT'])]
        back_set = valid_and_test[~valid_and_test['TRADE_DT'].isin(Calender_front['TRADE_DT'])]
        back_set.index = [i for i in range(len(back_set))]
        # merge train_set with front_set
        train_set = pd.concat([train_set, front_set], ignore_index=True)
        # up_date validation set to back_set
        data_append_validation = back_set
        del valid_and_test, front_set, back_set


    else:
        train_set = pd.concat([train_set, data_append], ignore_index=True)

    # 清内存 (可以考虑提前清理 train_x,train_y)
    del train_x, train_y, test_x

        #记录最优参数
    # best_params = best_model.get_params()
    # best_params_str = "_".join(f"{k}={v}" for k, v in best_params.items() if k in param_grid)


    # 把测试集拼入训练集中作为下一次的训练集 （这是没有验证集的情况）
    # train_set = pd.concat([train_set, data_append], ignore_index=True)



    # add a test set for one year, and the one add first above is validation set


    if rolling_style:
        Calender_drop = get_next_date(file_names, train_set.TRADE_DT[0].strftime('%Y%m%d'), get_date_unit)
        Calender_drop['TRADE_DT'] = pd.to_datetime(Calender_drop['TRADE_DT'], format='%Y%m%d')
        train_set = train_set[~train_set['TRADE_DT'].isin(Calender_drop['TRADE_DT'])]
        train_set.index = [i for i in range(len(train_set))]



# 拼接预测结果    
y_predict_df = pd.concat(test_y_predict_list, ignore_index=True)
y_predict_df



读取数据...: 100%|██████████| 240/240 [01:09<00:00,  3.47it/s]


训练从2016-01-04到2018-12-28的模型
训练从2019-01-02到2019-01-29的 valid 模型
This is 1 th round
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000000
[LightGBM] [Debug] init for col-wise cost 0.002885 seconds, init for row-wise cost 0.461779 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.593752 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28305
[LightGBM] [Info] Number of data points in the train set: 2444448, number of used features: 111
[LightGBM] [Info] Start training from score -0.000141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 7 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 7 an

读取数据...: 100%|██████████| 240/240 [01:16<00:00,  3.12it/s]


训练从2016-12-27到2019-12-25的模型
训练从2019-12-26到2020-01-23的 valid 模型
This is 1 th round
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000000
[LightGBM] [Debug] init for col-wise cost 0.000276 seconds, init for row-wise cost 0.404170 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.574751 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28305
[LightGBM] [Info] Number of data points in the train set: 2645413, number of used features: 111
[LightGBM] [Info] Start training from score -0.000036
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM]

读取数据...: 100%|██████████| 240/240 [01:23<00:00,  2.88it/s]


训练从2017-12-20到2020-12-22的模型
训练从2020-12-23到2021-01-20的 valid 模型
This is 1 th round
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000000
[LightGBM] [Debug] init for col-wise cost 0.000385 seconds, init for row-wise cost 0.488029 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.622171 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28305
[LightGBM] [Info] Number of data points in the train set: 2811950, number of used features: 111
[LightGBM] [Info] Start training from score 0.003570
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 7 and depth = 3
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 7 and depth = 3
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with lea

,TRADE_DT,S_INFO_WINDCODE,Amount_20D_AVG,Amt2RealizedVolatility_5D,BP,CFP_ttm,CashDividendPayoutRatio,EBIT2EV,EPFY3,EP_ded_qfa,...,or_fy3_tb_120d,rank_pbroe_qfa,rank_peg_qfa2,rounte_mom,rtd,turnover1m,turnover3m,turnover_std_3m,varTradeVolume5_10,predicted_y
0,2019-01-30,000001.SZ,0.043694,0.002485,0.004871,0.037860,-0.033200,0.009854,-0.011826,0.024651,...,-0.009313,-0.007964,0.007334,0.011431,-0.010390,-0.020304,-0.007206,-0.002296,0.025118,-0.001608
1,2019-01-30,000002.SZ,0.046027,0.015797,-0.030397,0.012712,0.014660,0.025622,-0.018617,-0.008623,...,0.001821,-0.004565,0.015289,0.008930,0.008016,-0.002926,-0.001588,0.003371,0.019819,-0.002711
2,2019-01-30,000004.SZ,-0.026611,-0.018814,-0.017500,0.000001,-0.011452,-0.004221,-0.001483,0.012576,...,-0.000089,-0.037702,0.003878,0.006902,0.009842,-0.001514,-0.014743,-0.006913,0.009518,-0.000115
3,2019-01-30,000005.SZ,0.023593,0.003291,-0.007984,-0.011775,-0.011305,0.012751,0.000463,0.002910,...,0.000201,-0.029763,-0.034087,-0.013565,0.007271,0.000007,-0.015479,0.007370,0.017462,-0.002711
4,2019-01-30,000006.SZ,0.013612,0.016984,-0.012417,-0.002931,0.018786,0.024067,-0.000171,-0.031150,...,-0.001369,0.031571,-0.014952,0.014012,-0.004704,-0.007253,0.006883,-0.003307,-0.007468,-0.002675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600127,2022-01-11,873169.BJ,-0.000710,0.000539,-0.003949,-0.000613,-0.005894,0.001651,-0.000733,0.000487,...,-0.001082,0.003813,0.003060,0.000421,-0.000143,-0.003552,-0.004465,-0.002607,0.000678,0.010223
600128,2022-01-11,873223.BJ,-0.000717,0.000545,-0.003983,-0.000616,-0.005945,0.001666,-0.000737,0.000491,...,-0.001090,0.003860,0.003092,0.000427,-0.000143,-0.003580,-0.004505,-0.002622,0.000679,0.010223
600129,2022-01-11,873305.BJ,-0.000717,0.000545,-0.003983,-0.000616,-0.005945,0.001666,-0.000737,0.000491,...,-0.001090,0.003860,0.003092,0.000427,-0.000143,-0.003580,-0.004505,-0.002622,0.000679,0.010223
600130,2022-01-11,873339.BJ,-0.000717,0.000545,-0.003983,-0.000616,-0.005945,0.001666,-0.000737,0.000491,...,-0.001090,0.003860,0.003092,0.000427,-0.000143,-0.003580,-0.004505,-0.002622,0.000679,0.010223


### 分年度多头组绩效回测: 

In [ ]:
# y_predict_df.to_pickle(y_predict_name)
y_predict_name = '_4_0.2_400.pkl'
y_predict_df = pd.read_pickle(y_predict_name)
grouped = y_predict_df.groupby(y_predict_df['TRADE_DT'].dt.year)

# 将每个年份的DataFrame存储在一个列表中
result = [group for _, group in grouped]

In [ ]:
y_predict_df[['TRADE_DT', 'S_INFO_WINDCODE', 'SUE_drift_excl_qfa', 'predicted_y']] #why SUE_drift_excl_qfa ???

### 全程多头组绩效回测

In [ ]:
# Factor = y_predict_df[['TRADE_DT', 'S_INFO_WINDCODE', 'SUE_drift_excl_qfa']]  #why SUE_drift_excl_qfa ???
# Factor = Factor.rename(columns={'SUE_drift_excl_qfa': 'FACTOR'})


Factor = y_predict_df[['TRADE_DT', 'S_INFO_WINDCODE', 'predicted_y']]  # try pred_y
Factor = Factor.rename(columns={'predicted_y': 'FACTOR'})

Factor['TRADE_DT'] = pd.to_datetime(Factor['TRADE_DT'], format='%Y/%m/%d', errors='coerce')
Factor['TRADE_DT'] = Factor['TRADE_DT'].dt.strftime('%Y%m%d')

Calender_all = pd.DataFrame({'TRADE_DT': Factor.TRADE_DT.unique()})
Risk_free_file = 'Input/副本无风险利率.xlsx'  # 无风险利率数据路径
RiskFreeReturn = LoadRiskFreeReturn(Risk_free_file, Calender_all)  # 加载无风险利率数据
Price, MarketCap, Industry, ST, Suspend, ListDate = Load_SQLData(conn, Calender_all, False, OutputPath_result,
                                                                 local=data_local)  # 加载SQL数据
BenchReturn = Load_Return(conn, Calender_all)  # 加载基准收益

Factor_all = Cal_Sift(Factor, ST, Suspend, ListDate)
GroupNum = 10
Factor_Group = Cal_Stratify(Factor_all, GroupNum)
# 获取分组持仓的股票池
Hold = Factor_Group[Factor_Group['GROUP'] == str(1)][['TRADE_DT', 'S_INFO_WINDCODE']].reset_index(drop=True)
# 根据持仓股票池，生成仓位数据
Position = Cal_Position(Hold)
Position['date'] = pd.to_datetime(Position['TRADE_DT'], format='%Y%m%d')
# 调用回测函数
NetValueTmp, PerformTmp = Back_Testing(Position, Price, BenchReturn, RiskFreeReturn)
NetValueTmp

In [ ]:
NetValueTmp

In [ ]:
PerformTmp

### 分年度多头组绩效回测

In [ ]:
NetValueTmp.index = pd.to_datetime(NetValueTmp.index, format='%Y%m%d')
grouped = NetValueTmp.groupby(NetValueTmp.index.year)
df_list = [(_, group) for _, group in grouped]
year_list = [year for year, group in grouped]
Perform = pd.DataFrame(columns=['年化收益(%)', '基准年化收益(%)', '超额年化收益(%)',
                                '超额年化波动(%)', '信息比率', '超额最大回撤(%)', '胜率(%)', '换手率(年均)'], index=year_list)

In [ ]:
for year, group in df_list:
    Return = pd.DataFrame(group[['策略净值', '策略收益']])
    Return.rename(columns={'策略净值': 'NETVALUE'}, inplace=True)
    Return.rename(columns={'策略收益': 'RETURN'}, inplace=True)
    Position_year = Position[Position['date'].dt.year == year]
    
    Calender = pd.DataFrame({'TRADE_DT': [timestamp.strftime('%Y%m%d') for timestamp in group.index]})
    Risk_free_file = 'Input/副本无风险利率.xlsx'  # 无风险利率数据路径
    RiskFreeReturn = LoadRiskFreeReturn(Risk_free_file, Calender)  # 加载无风险利率数据
    BenchReturn = Load_Return(conn, Calender)
    
    # 换手率
    TurnOverRate = Cal_TurnOver(Position_year)
    # 无风险收益：
    RiskfreeReturn, RiskfreeIntervalRet, RiskfreeIntervaStd, RiskfreeAnnualRet, RiskfreeAnnualStd, RiskfreeMaxdrawdown = PerfStatis(
        RiskFreeReturn, frequency='D')
    
    # 基准收益：
    # BenchReturn.loc[BenchReturn.index <= PeriodList[0], 'RETURN'] = 0 #这一句是否需要删除，有待验证
    BenchReturn, BenchIntervalRet, BenchIntervaStd, BenchAnnualRet, BenchAnnualStd, BenchMaxdrawdown = PerfStatis(
        BenchReturn, frequency='D')
    
    # 策略收益：
    StrategyReturn, StrategyIntervalRet, StrategyIntervaStd, StrategyAnnualRet, StrategyAnnualStd, StrategyMaxdrawdown = PerfStatis(
        Return, frequency='D', if_Returned=False)
    
    # 超额收益：
    initial_ExcessReturn = Cal_Excess(StrategyReturn, BenchReturn)
    ExcessReturn, ExcessIntervalRet, ExcessIntervaStd, ExcessAnnualRet, ExcessAnnualStd, ExcessMaxdrawdown = PerfStatis(
        initial_ExcessReturn, frequency='D', if_Returned=False)
    
    # 胜率：
    StrategyWinper = Cal_Winper(ExcessReturn)
    # 汇总：
    Perform.loc[year, :] = StrategyAnnualRet, BenchAnnualRet, ExcessAnnualRet, ExcessAnnualStd, (
        ExcessAnnualRet-RiskfreeAnnualRet)/ExcessAnnualStd, ExcessMaxdrawdown, StrategyWinper, TurnOverRate


# 同全程汇总
new_row = pd.DataFrame(PerformTmp[['年化收益(%)', '基准年化收益(%)', '超额年化收益(%)',
                       '超额年化波动(%)', '信息比率', '超额最大回撤(%)', '胜率(%)', '换手率(年均)']].values, index=['整体'])
new_row.columns = Perform.columns



Perform = pd.concat([Perform, new_row])
Perform
    

In [ ]:
NetValueTmp['超额回撤'] = 0  # 初始化最大回撤列为0
max_drawdown = 0  # 当前的最大回撤值
for i in range(len(NetValueTmp)):
    max_value = NetValueTmp['相对净值'][:i+1].max()  # 从第一天到当前天的最大净值
    drawdown = 100 * (NetValueTmp['相对净值'][i] / max_value - 1)  # 当前天的回撤
    max_drawdown = min(max_drawdown, drawdown)  # 更新当前的最大回撤值
    NetValueTmp['超额回撤'].iloc[i] = max_drawdown

del NetValueTmp['策略收益']
NetValueTmp

### 结果存储及净值曲线绘制

In [ ]:
# excel文件名称可能需要手动更改
writer = pd.ExcelWriter(OutputPath_result + '/' + result_excel)

# 将DataFrame写入不同的sheet
NetValueTmp.index = NetValueTmp.index.strftime('%Y-%m-%d')
Perform.to_excel(writer, sheet_name='多头组绩效', index=True)
NetValueTmp.to_excel(writer, sheet_name='多头组全程净值', index=True)

# 保存Excel文件
writer.save()

# 关闭ExcelWriter对象
writer.close()

In [ ]:
plt.figure(figsize=(30, 15))  # 设置图形大小



# 遍历DataFrame的每一列，为每一列绘制一条线，并设置标签为列名
for column in NetValueTmp.columns:
    plt.plot(NetValueTmp[column], label=column)

plt.xlabel('日期')  # 设置x轴标签
plt.ylabel('净值')  # 设置y轴标签
plt.title('分组超额折线图')  # 设置标题
plt.legend(loc='upper left', fontsize='x-large')  # 显示图例

x = np.arange(0, len(NetValueTmp.index), 100)  # 每100个数据点显示一个刻度


locs, labels = plt.xticks(NetValueTmp.index[x], rotation=45)  # 设置x轴刻度标签并旋转45度




plt.show()

### 组合优化

In [ ]:
# y_predict_df.to_pickle(y_predict_name)
y_predict_name = '_4_0.2_400.pkl'
y_predict_df = pd.read_pickle(y_predict_name)
grouped = y_predict_df.groupby(y_predict_df['TRADE_DT'].dt.year)

# 将每个年份的DataFrame存储在一个列表中
result = [group for _, group in grouped]

In [ ]:
y_predict_df['SUE_drift_excl_qfa']

In [ ]:
Factor = y_predict_df[['TRADE_DT', 'S_INFO_WINDCODE', 'SUE_drift_excl_qfa']]
Factor = Factor.rename(columns={'SUE_drift_excl_qfa': 'FACTOR'})
Factor['TRADE_DT'] = pd.to_datetime(
    Factor['TRADE_DT'], format='%Y/%m/%d', errors='coerce')
Factor['TRADE_DT'] = Factor['TRADE_DT'].dt.strftime('%Y%m%d')

Calender_all = pd.DataFrame({'TRADE_DT': Factor.TRADE_DT.unique()})
Risk_free_file = 'Input/副本无风险利率.xlsx'  # 无风险利率数据路径
RiskFreeReturn = LoadRiskFreeReturn(Risk_free_file, Calender_all)  # 加载无风险利率数据
Price, MarketCap, Industry, ST, Suspend, ListDate = Load_SQLData(conn, Calender_all, False, OutputPath_result,
                                                                 local=data_local)  # 加载SQL数据
# Weight = pd.read_pickle(r'D:\Microsoft VS Code\Code_repository\GuoJun\model_output_all\Weight.pkl')
Weight = pd.read_csv(
    r'D:\新建文件夹 (4)\WeChat Files\wxid_o6h4331nu3f012\FileStorage\File\2023-12\zscfgday000905.csv')
Weight = Weight[['trade_dt', 's_info_windcode', 'i_weight']].rename(
    columns={'trade_dt': 'TRADE_DT', 's_info_windcode': 'S_INFO_WINDCODE', 'i_weight': 'ADJ_I_WEIGHT'})
Weight = Weight[Weight['ADJ_I_WEIGHT'].notnull()]
Weight.index = range(len(Weight))
Weight.ADJ_I_WEIGHT = Weight.ADJ_I_WEIGHT/100
Weight['TRADE_DT'] = Weight['TRADE_DT'].str.replace('-', '')
BenchReturn = Load_Return(conn, Calender_all)  # 加载基准收益

Factor_all = Cal_Sift(Factor, ST, Suspend, ListDate)

In [ ]:
# Position_Optimal = Factor_all[['TRADE_DT', 'S_INFO_WINDCODE']]
# Position_Optimal['POSITION'] = np.zeros(len(Position_Optimal))
Position_list = []
Error_date_list = []
drop_list = []

Industry_list = Industry.IND_CODE.unique().tolist()
i = 0
# 遍历每一个交易日
for Date in tqdm(Factor_all.TRADE_DT.unique()[1:-40]):
    
    # 取行业代码和因子值
    Industry_list = Industry.IND_CODE.unique().tolist()
    Factor_Day = Factor_all[Factor_all['TRADE_DT'] == Date]
    Factor_Day.index = range(len(Factor_Day))
    
    # 在2023年后，存在部分股票的行业不存在于industry，删除这部分股票
    for index, code in enumerate(Factor_Day['S_INFO_WINDCODE']):
        if len(Industry.loc[Industry['S_INFO_WINDCODE'] == code]) == 0:
            drop_list.append(code)
    Factor_Day = Factor_Day[~Factor_Day['S_INFO_WINDCODE'].isin(drop_list)]
    
    # 取因子值
    # MU = Factor_all[Factor_all['TRADE_DT'] == Date]
    MU = pd.DataFrame(Factor_Day['FACTOR'].values)
    
    # 计算行业暴露，构造行业中性约束
    beq = pd.DataFrame(0, index=range(len(Industry_list)), columns=[0])
    Aeq = pd.DataFrame(0, index=range(len(Industry_list)),
                   columns=range(len(Factor_Day)))
    for index, code in enumerate(Factor_Day['S_INFO_WINDCODE']):
        ind = Industry.loc[Industry['S_INFO_WINDCODE']
                       == code, 'IND_CODE'].values[0]
        Aeq.iloc[Industry_list.index(ind), index] = 1
    
    # 计算市值暴露，构造市值中性约束
    MarketCap_Day = MarketCap[MarketCap['TRADE_DT'] == Date]
    MarketCap_Day = MarketCap_Day[MarketCap_Day.S_INFO_WINDCODE.isin(
        Factor_Day.S_INFO_WINDCODE.tolist())]
    log_market_value = np.log(MarketCap_Day['S_VAL_MV'])
    mean = np.sum(log_market_value *
              MarketCap_Day['S_VAL_MV']) / np.sum(MarketCap_Day['S_VAL_MV'])

    std = MarketCap_Day['S_VAL_MV'].std()
    MarketCap_Day['VALMV_Expose'] = (log_market_value - mean) / std

    A_leq = pd.concat([pd.DataFrame(MarketCap_Day.VALMV_Expose.to_numpy()).T,
                   pd.DataFrame(-MarketCap_Day.VALMV_Expose.to_numpy()).T])

    b_leq = pd.DataFrame([0.5, 0.5])
    
    # Weight_Day['ADJ_I_WEIGHT'] = Weight_Day['ADJ_I_WEIGHT']*100
    # 取基准指数权重
    Weight_Day = Weight[Weight['TRADE_DT'] == Date]
    bmw_Day = pd.DataFrame(
        {'TRADE_DT': Factor_Day.TRADE_DT, 'S_INFO_WINDCODE': Factor_Day.S_INFO_WINDCODE, 'Weight': np.zeros(len(Factor_Day))})  # 基准指数权重

    for code in bmw_Day.S_INFO_WINDCODE:
        if code in Weight_Day.S_INFO_WINDCODE.tolist():
            bmw_Day.loc[bmw_Day.S_INFO_WINDCODE == code,
                        'Weight'] = Weight_Day.loc[Weight_Day.S_INFO_WINDCODE == code, 'ADJ_I_WEIGHT'].values
    
    Price_Day = Price[Price['TRADE_DT'] == Date]
    Price_Day = Price_Day[Price_Day.S_INFO_WINDCODE.isin(Factor_Day.S_INFO_WINDCODE.tolist())]
    Price_Day.index = range(len(Price_Day))
    
    # 初始主动权重为基准指数权重，否则为上一期的主动权重
    bmw = pd.DataFrame(bmw_Day.Weight.values).T
    if i == 0:
        w0 = bmw - bmw
    else:
        # w0 = pd.DataFrame(a11).T 
    # 取收盘价，用于计算每一期间权重变化
        
        Weight_new = Factor_Day.merge(Weight_final, on='S_INFO_WINDCODE',
                                    how='left')
        Weight_new['S_DQ_ADJCLOSE_NEW'] = Price_Day.S_DQ_ADJPRECLOSE.values
        Weight_new['New_Weight'] = Weight_new['Weight'] * (Weight_new['S_DQ_ADJCLOSE_NEW'] / Weight_new['S_DQ_ADJCLOSE'])
        Weight_new['normalized_weight'] = Weight_new['New_Weight'] / Weight_new['New_Weight'].sum()
        Weight_new.fillna(0, inplace=True)
        
        w0 = pd.DataFrame(Weight_new.normalized_weight.values - bmw_Day.Weight.values).T
    
    # 换手率约束
    turn = 100
    solver1 = 'glpk'
    
    # 优化函数，如果在某个换仓日优化问题无解，则这个日期会被存在error_date里，主动权重沿用上一期，有解时error_date是目标函数的最优值，主要用于看无解的日期
    a11, error_date = cvxlp2_turn(
        Date, MU, bmw, w0, turn, A_leq, b_leq, Aeq, beq, 0.01, solver1)
    # print(error_date)
    
    # 仓位存储
    Weight_final = pd.DataFrame({'TRADE_DT': Factor_Day.TRADE_DT,
                                 'S_INFO_WINDCODE': Factor_Day.S_INFO_WINDCODE, 'Weight': bmw_Day.Weight + a11, 'S_DQ_ADJCLOSE': Price_Day.S_DQ_ADJCLOSE})
    Position_Day = pd.DataFrame(
        {'TRADE_DT': bmw_Day.TRADE_DT, 'S_INFO_WINDCODE': bmw_Day.S_INFO_WINDCODE, 'POSITION': bmw_Day.Weight + a11})
    Position_list.append(Position_Day)
    Error_date_list.append(error_date)
    i = i+1
    print('换手率差值为'+str(np.sum(abs(pd.DataFrame(a11).T - w0), axis = 1)))
    
    

In [ ]:
# Position_list = []
# Error_date_list = []
drop_list = []

Industry_list = Industry.IND_CODE.unique().tolist()
i = 2
# 遍历每一个交易日
Date = Factor_all.TRADE_DT.unique()[i+1]
Industry_list = Industry.IND_CODE.unique().tolist()
Factor_Day = Factor_all[Factor_all['TRADE_DT'] == Date]
Factor_Day.index = range(len(Factor_Day))

# 在2023年后，存在部分股票的行业不存在于industry，删除这部分股票
for index, code in enumerate(Factor_Day['S_INFO_WINDCODE']):
    if len(Industry.loc[Industry['S_INFO_WINDCODE'] == code]) == 0:
        drop_list.append(code)
Factor_Day = Factor_Day[~Factor_Day['S_INFO_WINDCODE'].isin(drop_list)]

# 取因子值
# MU = Factor_all[Factor_all['TRADE_DT'] == Date]
MU = pd.DataFrame(Factor_Day['FACTOR'].values)

# 计算行业暴露，构造行业中性约束
beq = pd.DataFrame(0, index=range(len(Industry_list)), columns=[0])
Aeq = pd.DataFrame(0, index=range(len(Industry_list)),
                columns=range(len(Factor_Day)))
for index, code in enumerate(Factor_Day['S_INFO_WINDCODE']):
    ind = Industry.loc[Industry['S_INFO_WINDCODE']
                    == code, 'IND_CODE'].values[0]
    Aeq.iloc[Industry_list.index(ind), index] = 1

# 计算市值暴露，构造市值中性约束
MarketCap_Day = MarketCap[MarketCap['TRADE_DT'] == Date]
MarketCap_Day = MarketCap_Day[MarketCap_Day.S_INFO_WINDCODE.isin(
    Factor_Day.S_INFO_WINDCODE.tolist())]
log_market_value = np.log(MarketCap_Day['S_VAL_MV'])
mean = np.sum(log_market_value *
            MarketCap_Day['S_VAL_MV']) / np.sum(MarketCap_Day['S_VAL_MV'])

std = MarketCap_Day['S_VAL_MV'].std()
MarketCap_Day['VALMV_Expose'] = (log_market_value - mean) / std

A_leq = pd.concat([pd.DataFrame(MarketCap_Day.VALMV_Expose.to_numpy()).T,
                pd.DataFrame(-MarketCap_Day.VALMV_Expose.to_numpy()).T])

b_leq = pd.DataFrame([0.5, 0.5])

# 取基准指数权重
Weight_Day = Weight[Weight['TRADE_DT'] == Date]
bmw_Day = pd.DataFrame(
    {'TRADE_DT': Factor_Day.TRADE_DT, 'S_INFO_WINDCODE': Factor_Day.S_INFO_WINDCODE, 'Weight': np.zeros(len(Factor_Day))})  # 基准指数权重

for code in bmw_Day.S_INFO_WINDCODE:
    if code in Weight_Day.S_INFO_WINDCODE.tolist():
        bmw_Day.loc[bmw_Day.S_INFO_WINDCODE == code,
                    'Weight'] = Weight_Day.loc[Weight_Day.S_INFO_WINDCODE == code, 'ADJ_I_WEIGHT'].values

# 初始主动权重为基准指数权重，否则为上一期的主动权重
bmw = pd.DataFrame(bmw_Day.Weight.values).T
Price_Day = Price[Price['TRADE_DT'] == Date]

if i == 0:
    w0 = bmw
else:
    # w0 = pd.DataFrame(a11).T 
    # 取收盘价，用于计算每一期间权重变化
    Price_Day = Price_Day[Price_Day.S_INFO_WINDCODE.isin(Factor_Day.S_INFO_WINDCODE.tolist())]
    Price_Day.index = range(len(Price_Day))
    Weight_new = Factor_Day.merge(Weight_final, on='S_INFO_WINDCODE',
                                how='left')
    Weight_new['S_DQ_ADJCLOSE_NEW'] = Price_Day.S_DQ_ADJPRECLOSE.values
    Weight_new['New_Weight'] = Weight_new['Weight'] * (Weight_new['S_DQ_ADJCLOSE_NEW'] / Weight_new['S_DQ_ADJCLOSE'])
    Weight_new['normalized_weight'] = Weight_new['New_Weight'] / Weight_new['New_Weight'].sum()
    Weight_new.fillna(0, inplace=True)
    
    w0 = pd.DataFrame(Weight_new.normalized_weight.values - bmw_Day.Weight.values).T

# 换手率约束
turn = 1
solver1 = 'glpk'

# 优化函数，如果在某个换仓日优化问题无解，则这个日期会被存在error_date里，主动权重沿用上一期，有解时error_date是目标函数的最优值，主要用于看无解的日期
a11, error_date = cvxlp2_turn(
    Date, MU, bmw, w0, turn, A_leq, b_leq, Aeq, beq, 0.01, solver1)

# 记录收盘价用于计算权重
Weight_final = pd.DataFrame({'TRADE_DT': Factor_Day.TRADE_DT,
                            'S_INFO_WINDCODE': Factor_Day.S_INFO_WINDCODE, 'Weight': bmw_Day.Weight + a11, 'S_DQ_ADJCLOSE': Price_Day.S_DQ_ADJCLOSE})
Position_Day = pd.DataFrame(
    {'TRADE_DT': bmw_Day.TRADE_DT, 'S_INFO_WINDCODE': bmw_Day.S_INFO_WINDCODE, 'POSITION': bmw_Day.Weight + a11})
# print(error_date)
Position_Day

In [ ]:
Aeq


In [ ]:
drop_list = []

Industry_list = Industry.IND_CODE.unique().tolist()
i = 0
# 遍历每一个交易日
Date = Factor_all.TRADE_DT.unique()[i+1]
Industry_list = Industry.IND_CODE.unique().tolist()
Factor_Day = Factor_all[Factor_all['TRADE_DT'] == Date]
Factor_Day.index = range(len(Factor_Day))

# 在2023年后，存在部分股票的行业不存在于industry，删除这部分股票
for index, code in enumerate(Factor_Day['S_INFO_WINDCODE']):
    if len(Industry.loc[Industry['S_INFO_WINDCODE'] == code]) == 0:
        drop_list.append(code)
Factor_Day = Factor_Day[~Factor_Day['S_INFO_WINDCODE'].isin(drop_list)]

# 取因子值
# MU = Factor_all[Factor_all['TRADE_DT'] == Date]
MU = pd.DataFrame(Factor_Day['FACTOR'].values)

# 计算行业暴露，构造行业中性约束
beq = pd.DataFrame(0, index=range(len(Industry_list)), columns=[0])
Aeq = pd.DataFrame(0, index=range(len(Industry_list)),
                   columns=range(len(Factor_Day)))
for index, code in enumerate(Factor_Day['S_INFO_WINDCODE']):
    ind = Industry.loc[Industry['S_INFO_WINDCODE']
                       == code, 'IND_CODE'].values[0]
    Aeq.iloc[Industry_list.index(ind), index] = 1

# 计算市值暴露，构造市值中性约束
MarketCap_Day = MarketCap[MarketCap['TRADE_DT'] == Date]
MarketCap_Day = MarketCap_Day[MarketCap_Day.S_INFO_WINDCODE.isin(
    Factor_Day.S_INFO_WINDCODE.tolist())]
log_market_value = np.log(MarketCap_Day['S_VAL_MV'])
mean = np.sum(log_market_value *
              MarketCap_Day['S_VAL_MV']) / np.sum(MarketCap_Day['S_VAL_MV'])

std = MarketCap_Day['S_VAL_MV'].std()
MarketCap_Day['VALMV_Expose'] = (log_market_value - mean) / std

A_leq = pd.concat([pd.DataFrame(MarketCap_Day.VALMV_Expose.to_numpy()).T,
                   pd.DataFrame(-MarketCap_Day.VALMV_Expose.to_numpy()).T])

b_leq = pd.DataFrame([0.5, 0.5])

# 取基准指数权重
Weight_Day = Weight[Weight['TRADE_DT'] == Date]
bmw_Day = pd.DataFrame(
    {'TRADE_DT': Factor_Day.TRADE_DT, 'S_INFO_WINDCODE': Factor_Day.S_INFO_WINDCODE, 'Weight': np.zeros(len(Factor_Day))})  # 基准指数权重

for code in bmw_Day.S_INFO_WINDCODE:
    if code in Weight_Day.S_INFO_WINDCODE.tolist():
        bmw_Day.loc[bmw_Day.S_INFO_WINDCODE == code,
                    'Weight'] = Weight_Day.loc[Weight_Day.S_INFO_WINDCODE == code, 'ADJ_I_WEIGHT'].values

# 初始主动权重为基准指数权重，否则为上一期的主动权重
bmw = pd.DataFrame(bmw_Day.Weight.values).T

if i == 0:
    w0 = bmw
else:
    # w0 = pd.DataFrame(a11).T
    # 取收盘价，用于计算每一期间权重变化
    Price_Day = Price[Price['TRADE_DT'] == Date]
    Price_Day = Price_Day[Price_Day.S_INFO_WINDCODE.isin(
        Factor_Day.S_INFO_WINDCODE.tolist())]
    Price_Day.index = range(len(Price_Day))
    Weight_new = Factor_Day.merge(Weight_final, on='S_INFO_WINDCODE',
                                  how='left')
    Weight_new['S_DQ_ADJCLOSE_NEW'] = Price_Day.S_DQ_ADJPRECLOSE.values
    Weight_new['New_Weight'] = Weight_new['Weight'] * \
        (Weight_new['S_DQ_ADJCLOSE_NEW'] / Weight_new['S_DQ_ADJCLOSE'])
    Weight_new['normalized_weight'] = Weight_new['New_Weight'] / \
        Weight_new['New_Weight'].sum()
    Weight_new.fillna(0, inplace=True)

In [ ]:
Weight_new = Factor_Day.merge(Weight_final, on='S_INFO_WINDCODE',
                 how='left')
Weight_new['S_DQ_ADJCLOSE_NEW'] = Price_Day.S_DQ_ADJPRECLOSE.values
Weight_new['New_Weight'] = Weight_new['Weight'] * \
    (Weight_new['S_DQ_ADJCLOSE_NEW'] / Weight_new['S_DQ_ADJCLOSE'])
Weight_new['normalized_weight'] = Weight_new['New_Weight'] / \
    Weight_new['New_Weight'].sum()
Weight_new.fillna(0, inplace = True)
Weight_new

In [ ]:
# Weight_new['price_change_ratio'] = (
#     Weight_new['S_DQ_ADJCLOSE_NEW'] / Weight_new['S_DQ_ADJCLOSE'])

# Calculate the new weighted column
Weight_new['New_Weight'] = Weight_new['Weight'] * \
     (Weight_new['S_DQ_ADJCLOSE_NEW'] / Weight_new['S_DQ_ADJCLOSE'])

# Normalize the new weighted column
Weight_new['normalized_weight'] = Weight_new['New_Weight'] / \
    Weight_new['New_Weight'].sum()

In [ ]:
Weight_new.fillna(0, inplace = True)
Weight_new

### 得到优化后权重进行全程和分年度回测

In [ ]:
Position = pd.concat(Position_list, ignore_index = True)
Position

In [ ]:
Position = Position[Position.POSITION != 0]

In [ ]:
BenchReturn = BenchReturn.loc[BenchReturn.index < max(Position.TRADE_DT)]
Price = Price.loc[Price.TRADE_DT < max(Position.TRADE_DT)]
Position['date'] = pd.to_datetime(Position['TRADE_DT'], format='%Y%m%d')
# 全程回测
NetValueTmp, PerformTmp = Back_Testing(
    Position, Price, BenchReturn, RiskFreeReturn)
NetValueTmp.index = pd.to_datetime(NetValueTmp.index, format='%Y%m%d')
grouped = NetValueTmp.groupby(NetValueTmp.index.year)
df_list = [(_, group) for _, group in grouped]
year_list = [year for year, group in grouped]
Perform = pd.DataFrame(columns=['年化收益(%)', '基准年化收益(%)', '超额年化收益(%)',
                                '超额年化波动(%)', '信息比率', '超额最大回撤(%)', '胜率(%)', '换手率(年均)'], index=year_list)

# 分年度回测
for year, group in df_list:
    Return = pd.DataFrame(group[['策略净值', '策略收益']])
    Return.rename(columns={'策略净值': 'NETVALUE'}, inplace=True)
    Return.rename(columns={'策略收益': 'RETURN'}, inplace=True)
    Position_year = Position[Position['date'].dt.year == year]

    Calender = pd.DataFrame(
        {'TRADE_DT': [timestamp.strftime('%Y%m%d') for timestamp in group.index]})
    Risk_free_file = 'Input/副本无风险利率.xlsx'  # 无风险利率数据路径
    RiskFreeReturn = LoadRiskFreeReturn(Risk_free_file, Calender)  # 加载无风险利率数据
    BenchReturn = Load_Return(conn, Calender)

    # 换手率
    TurnOverRate = Cal_TurnOver(Position_year)
    # 无风险收益：
    RiskfreeReturn, RiskfreeIntervalRet, RiskfreeIntervaStd, RiskfreeAnnualRet, RiskfreeAnnualStd, RiskfreeMaxdrawdown = PerfStatis(
        RiskFreeReturn, frequency='D')

    # 基准收益：
    # BenchReturn.loc[BenchReturn.index <= PeriodList[0], 'RETURN'] = 0 #这一句是否需要删除，有待验证
    BenchReturn, BenchIntervalRet, BenchIntervaStd, BenchAnnualRet, BenchAnnualStd, BenchMaxdrawdown = PerfStatis(
        BenchReturn, frequency='D')

    # 策略收益：
    StrategyReturn, StrategyIntervalRet, StrategyIntervaStd, StrategyAnnualRet, StrategyAnnualStd, StrategyMaxdrawdown = PerfStatis(
        Return, frequency='D', if_Returned=False)

    # 超额收益：
    initial_ExcessReturn = Cal_Excess(StrategyReturn, BenchReturn)
    ExcessReturn, ExcessIntervalRet, ExcessIntervaStd, ExcessAnnualRet, ExcessAnnualStd, ExcessMaxdrawdown = PerfStatis(
        initial_ExcessReturn, frequency='D', if_Returned=False)

    # 胜率：
    StrategyWinper = Cal_Winper(ExcessReturn)
    # 汇总：
    Perform.loc[year, :] = StrategyAnnualRet, BenchAnnualRet, ExcessAnnualRet, ExcessAnnualStd, (
        ExcessAnnualRet-RiskfreeAnnualRet)/ExcessAnnualStd, ExcessMaxdrawdown, StrategyWinper, TurnOverRate


# 同全程汇总
new_row = pd.DataFrame(PerformTmp[['年化收益(%)', '基准年化收益(%)', '超额年化收益(%)',
                       '超额年化波动(%)', '信息比率', '超额最大回撤(%)', '胜率(%)', '换手率(年均)']].values, index=['整体'])
new_row.columns = Perform.columns
Perform = pd.concat([Perform, new_row])
Perform

In [ ]:
NetValueTmp

In [ ]:
frequency = 'D'
PeriodList = Position.TRADE_DT.unique()
# 计算策略的日频收益率
# bins = np.insert(PeriodList, 0, '00000000')
# Price['TRADE_PERIOD'] = pd.cut(
#     Price['TRADE_DT'], bins=bins, labels=PeriodList[:])
Price['TRADE_PERIOD'] = [
    PeriodList[np.where(PeriodList >= i)[0][0]] for i in Price.TRADE_DT]
StrategyReturn = pd.DataFrame(Price.groupby(Price.TRADE_PERIOD).apply(
    M2D, Position).values, index=Price.TRADE_DT.unique(), columns=['RETURN'])
StrategyReturn.fillna(0, inplace=True)
NetValue = pd.DataFrame(index=BenchReturn.index,
                        columns=['基准净值', '策略净值', '相对净值', '策略收益'])
NetValue['策略收益'] = StrategyReturn['RETURN']
# 计算换手率
TurnOverRate = Cal_TurnOver(Position)
# 计算无风险利率（货基指数）的年化收益
RiskfreeReturn, RiskfreeIntervalRet, RiskfreeIntervaStd, RiskfreeAnnualRet, RiskfreeAnnualStd, RiskfreeMaxdrawdown = PerfStatis(
    RiskFreeReturn, frequency)
# 统计策略的净值、区间收益率、区间波动率、年化收益率、年化波动率、夏普比率、最大回撤
StrategyReturn, StrategyIntervalRet, StrategyIntervaStd, StrategyAnnualRet, StrategyAnnualStd, StrategyMaxdrawdown = PerfStatis(
    StrategyReturn, frequency)
# 统计基准的净值、区间收益率、区间波动率、年化收益率、年化波动率、夏普比率、最大回撤
BenchReturn.loc[BenchReturn.index <= PeriodList[0], 'RETURN'] = 0
BenchReturn, BenchIntervalRet, BenchIntervaStd, BenchAnnualRet, BenchAnnualStd, BenchMaxdrawdown = PerfStatis(
    BenchReturn, frequency)

In [ ]:
BenchReturn

In [ ]:
Position

In [ ]:
plt.figure(figsize=(30, 15))  # 设置图形大小
NetValueTmp.index = NetValueTmp.index.strftime('%Y-%m-%d')
# 遍历DataFrame的每一列，为每一列绘制一条线，并设置标签为列名
for column in NetValueTmp.columns:
    plt.plot(NetValueTmp[column], label=column)

plt.xlabel('日期')  # 设置x轴标签
plt.ylabel('净值')  # 设置y轴标签
plt.title('分组超额折线图')  # 设置标题
plt.legend(loc='upper left', fontsize='x-large')  # 显示图例

x = np.arange(0, len(NetValueTmp.index), 100)  # 每100个数据点显示一个刻度
plt.xticks(x, NetValueTmp.index[x], rotation=45)  # 设置x轴刻度标签并旋转45度

plt.show()